In [1]:
import os
import sys
import gc

import scipy.stats as stats
import scvi
import numpy as np
import pandas as pd
import scanpy as sc
from anndata import AnnData
from fast_matrix_market import mmread

import scipy
from scipy.spatial.distance import jensenshannon
from scipy.stats import pearsonr

import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib import rcParams

import torch
import cell2location

# Set matplotlib parameters
rcParams['pdf.fonttype'] = 42  # Enables correct plotting of text for PDFs

/bgfs/alee/LO_LAB/Personal/Brent_Schlegel/envs/CITEgeist/lib/python3.10/site-packages/scvi/_settings.py:63: UserWarning: Since v1.0.0, scvi-tools no longer uses a random seed by default. Run `scvi.settings.seed = 0` to reproduce results from previous versions.
  self.seed = seed
/bgfs/alee/LO_LAB/Personal/Brent_Schlegel/envs/CITEgeist/lib/python3.10/site-packages/scvi/_settings.py:70: UserWarning: Setting `dl_pin_memory_gpu_training` is deprecated in v1.0 and will be removed in v1.1. Please pass in `pin_memory` to the data loaders instead.
  self.dl_pin_memory_gpu_training = (
/bgfs/alee/LO_LAB/Personal/Brent_Schlegel/envs/CITEgeist/lib/python3.10/site-packages/anndata/utils.py:429: FutureWarning: Importing read_csv from `anndata` is deprecated. Import anndata.io.read_csv instead.
  warnings.warn(msg, FutureWarning)
/bgfs/alee/LO_LAB/Personal/Brent_Schlegel/envs/CITEgeist/lib/python3.10/site-packages/anndata/utils.py:429: FutureWarning: Importing read_loom from `anndata` is deprecated.

In [2]:
results_folder = '/bgfs/alee/LO_LAB/Personal/Brent_Schlegel/Projects/Wu_Visium/Simulations/larger_ref/cell2location/high_seg'

# create paths and names to results folders for reference regression and cell2location models
ref_run_name = '/bgfs/alee/LO_LAB/Personal/Brent_Schlegel/Projects/Wu_Visium/Simulations/cell2loc_test2/reference_signatures_test2'
run_name = f'{results_folder}/cell2location_map'

adata_file = f"{ref_run_name}/reference_major.h5ad"
adata_ref = sc.read_h5ad(adata_file)
mod = cell2location.models.RegressionModel.load(f"{ref_run_name}", adata_ref)

# view anndata_setup as a sanity check
mod.view_anndata_setup()

INFO     File                                                                                                      
         /bgfs/alee/LO_LAB/Personal/Brent_Schlegel/Projects/Wu_Visium/Simulations/cell2loc_test2/reference_signatur
         es_test2/model.pt already downloaded                                                                      


/bgfs/alee/LO_LAB/Personal/Brent_Schlegel/envs/CITEgeist/lib/python3.10/site-packages/scvi/model/base/_utils.py:68: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = tor

Epoch 1/523:   0%|          | 1/523 [00:01<15:02,  1.73s/it, v_num=1]

`Trainer.fit` stopped: `max_steps=1` reached.


Epoch 1/523:   0%|          | 1/523 [00:01<15:55,  1.83s/it, v_num=1]


...

         Summary Statistics         
┏━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━┓
┃     Summary Stat Key     ┃ Value ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━┩
│         n_batch          │  11   │
│         n_cells          │ 38241 │
│ n_extra_categorical_covs │   1   │
│ n_extra_continuous_covs  │   0   │
│         n_labels         │   9   │
│          n_vars          │ 13048 │
└──────────────────────────┴───────┘

                             Data Registry                             
┏━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃      Registry Key      ┃            scvi-tools Location             ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│           X            │                  adata.X                   │
│         batch          │          adata.obs['_scvi_batch']          │
│ extra_categorical_covs │ adata.obsm['_scvi_extra_categorical_covs'] │
│         ind_x          │           adata.obs['_indices']            │
│         labels         │         adata.obs['_scvi_labels']          │
└────────────────────────┴────────────────────────────────────────────┘

                     batch State Registry                     
┏━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┓
┃     Source Location     ┃ Categories ┃ scvi-tools Encoding ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━┩
│ adata.obs['patient_ID'] │  CID3941   │          0          │
│                         │  CID3948   │          1          │
│                         │  CID4040   │          2          │
│                         │  CID4067   │          3          │
│                         │  CID4290A  │          4          │
│                         │  CID4398   │          5          │
│                         │  CID4461   │          6          │
│                         │  CID4463   │          7          │
│                         │  CID4471   │          8          │
│                         │  CID4530N  │          9          │
│                         │  CID4535   │         10          │
└─────────────────────────┴────────────┴─────────────────────┘

                       labels State Registry                       
┏━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┓
┃    Source Location    ┃    Categories     ┃ scvi-tools Encoding ┃
┡━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━┩
│ adata.obs['celltype'] │      B-cells      │          0          │
│                       │       CAFs        │          1          │
│                       │ Cancer Epithelial │          2          │
│                       │    Endothelial    │          3          │
│                       │      Myeloid      │          4          │
│                       │ Normal Epithelial │          5          │
│                       │        PVL        │          6          │
│                       │   Plasmablasts    │          7          │
│                       │      T-cells      │          8          │
└───────────────────────┴───────────────────┴─────────────────────┘

            extra_categorical_covs State Registry             
┏━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┓
┃     Source Location     ┃ Categories ┃ scvi-tools Encoding ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━┩
│ adata.obs['patient_ID'] │  CID3941   │          0          │
│                         │  CID3948   │          1          │
│                         │  CID4040   │          2          │
│                         │  CID4067   │          3          │
│                         │  CID4290A  │          4          │
│                         │  CID4398   │          5          │
│                         │  CID4461   │          6          │
│                         │  CID4463   │          7          │
│                         │  CID4471   │          8          │
│                         │  CID4530N  │          9          │
│                         │  CID4535   │         10          │
│                         │            │                     │
└─────────────────────────┴────────────┴─────────────────────┘

In [4]:
# Prepare reference signatures
if 'means_per_cluster_mu_fg' in adata_ref.varm.keys():
    inf_aver = adata_ref.varm['means_per_cluster_mu_fg'][[f'means_per_cluster_mu_fg_{i}'
                                    for i in adata_ref.uns['mod']['factor_names']]].copy()
else:
    inf_aver = adata_ref.var[[f'means_per_cluster_mu_fg_{i}'
                                    for i in adata_ref.uns['mod']['factor_names']]].copy()
inf_aver.columns = adata_ref.uns['mod']['factor_names']
inf_aver.iloc[0:5, 0:5]

,B-cells,CAFs,Cancer Epithelial,Endothelial,Myeloid
Index,,,,,
FO538757.2,0.030329,0.115344,0.278675,0.146403,0.119704
AP006222.2,0.004011,0.096504,0.095929,0.081992,0.059953
SAMD11,0.003026,0.146523,0.005799,0.005670,0.001201
NOC2L,0.037645,0.126233,0.608349,0.197264,0.133015
RP11-54O7.17,0.002834,0.021926,0.007578,0.008690,0.025254


In [5]:
inf_aver.loc["CD3D"]

B-cells              0.002943
CAFs                 0.002788
Cancer Epithelial    0.010356
Endothelial          0.003185
Myeloid              0.005268
Normal Epithelial    0.003105
PVL                  0.002288
Plasmablasts         0.011862
T-cells              1.392241
Name: CD3D, dtype: float32

In [ ]:
# Directories and paths
input_folder  = "/bgfs/alee/LO_LAB/Personal/Brent_Schlegel/Projects/Wu_Visium/Simulations/larger_ref/replicates/high_seg/h5ad_objects"
output_folder = "/bgfs/alee/LO_LAB/Personal/Brent_Schlegel/Projects/Wu_Visium/Simulations/larger_ref/cell2location/high_seg"

# Iterate over all replicates
replicates = [f for f in os.listdir(input_folder) if f.endswith("GEX.h5ad")]

for replicate_name in replicates:
    replicate_path = os.path.join(input_folder, replicate_name)
    run_name = os.path.join(output_folder, f"cell2location_map_{replicate_name.split('_')[2]}")

    # Load Visium query dataset (replicate)
    adata_vis_0 = sc.read_h5ad(replicate_path)

    # Set "SYMBOL" variable
    adata_vis_0.var['SYMBOL'] = adata_vis_0.var.index
    adata_ref.var['SYMBOL'] = adata_ref.var.index

    # Find mitochondria-encoded (MT) genes
    adata_vis_0.var['MT_gene'] = [gene.startswith('MT-') for gene in adata_vis_0.var['SYMBOL']]

    # Remove MT genes for spatial mapping (keeping their counts in the object)
    adata_vis_0.obsm['MT'] = adata_vis_0[:, adata_vis_0.var['MT_gene'].values].X.toarray()
    adata_vis_0 = adata_vis_0[:, ~adata_vis_0.var['MT_gene'].values]
    
    # Prepare `adata_vis_0` for cell2location
    adata_vis_0.X_norm = adata_vis_0.X
    adata_vis_0.X = np.expm1(adata_vis_0.X_norm).round()

    # Find shared genes and subset
    intersect  = np.intersect1d(adata_vis_0.var_names, inf_aver.index)
    adata_vis_0 = adata_vis_0[:, intersect].copy()
    inf_aver   = inf_aver.loc[intersect, :].copy()

    #adata_vis_0 = adata_vis_0[:, adata_vis_0.var_names.isin(intersect)].copy()
    #inf_aver = inf_aver.loc[inf_aver.index.isin(intersect), :].copy()

    ## Ensure matching gene order
    #adata_vis_0.var_names = adata_vis_0.var_names.sort_values()
    #inf_aver = inf_aver.sort_index()

    # Setup cell2location
    cell2location.models.Cell2location.setup_anndata(adata=adata_vis_0)

   # create and train the model
    mod = cell2location.models.Cell2location(
        adata_vis_0, cell_state_df=inf_aver,
        # the expected average cell abundance: tissue-dependent
        # hyper-prior which can be estimated from paired histology:
        N_cells_per_location=5,
        # hyperparameter controlling normalisation of
        # within-experiment variation in RNA detection:
        detection_alpha=200
    )
    mod.view_anndata_setup()

    # train the model
    mod.train(max_epochs=30000,
          # train using full data (batch_size=None)
          batch_size=None,
          # use all data points in training because
          # we need to estimate cell abundance at all locations
          train_size=1,
          use_gpu=True,
         )

    # Export posterior and save results
    adata_vis_0 = mod.export_posterior(
        adata_vis_0, sample_kwargs={'num_samples': 3000, 'batch_size': mod.adata.n_obs, 'use_gpu': True}
    )
    mod.save(f"{run_name}", overwrite=True)

    # Compute expected expression per cell type
    expected_dict = mod.module.model.compute_expected_per_cell_type(
        mod.samples["post_sample_q05"], mod.adata_manager
    )

    # Add to anndata layers
    for i, n in enumerate(mod.factor_names_):
        adata_vis_0.layers[n] = expected_dict['mu'][i]

    # Add cell abundance quantile to `obs`
    adata_vis_0.obs[adata_vis_0.uns['mod']['factor_names']] = adata_vis_0.obsm['q05_cell_abundance_w_sf']

    # Save proportions
    df = adata_vis_0.obsm['q95_cell_abundance_w_sf']
    total_abundance = df.sum(axis=1)
    proportions = df.div(total_abundance, axis=0)
    proportions.columns = [col.split('_')[-1] for col in proportions.columns]
    proportions.reset_index(inplace=True)
    proportions.rename(columns={'index': 'spot'}, inplace=True)
    proportions.to_csv(f"{run_name}/cell2loc_deconv_predictions.csv", index=False)

    # Export layers to CSV
    layers_output_dir = f"{run_name}/layers"
    os.makedirs(layers_output_dir, exist_ok=True)
    for layer_name in adata_vis_0.layers.keys():
        layer_data = adata_vis_0.layers[layer_name].toarray()
        df = pd.DataFrame(layer_data, index=adata_vis_0.obs.index, columns=adata_vis_0.var.index)
        df.to_csv(f"{layers_output_dir}/{layer_name}_layer.csv")

    
    # Save AnnData with results
    adata_file = f"{run_name}/sp.h5ad"
    adata_vis_0.write(adata_file)
    
    # Generate and save plots
    fig_dir = f"{run_name}/plots"
    os.makedirs(fig_dir, exist_ok=True)

    # Extract a simplified replicate name
    simplified_name = os.path.splitext(os.path.basename(replicate_name))[0]

    # UMAP visualization
    sc.pl.embedding(
        adata_vis_0,
        basis="spatial",
        color=adata_vis_0.uns['mod']['factor_names'],
        cmap="magma",
        ncols=2,
        show=False,
        save=f"umap_{simplified_name}.pdf",
    )

    # Spatial abundance visualization
    for ct in adata_vis_0.uns['mod']['factor_names']:
        sc.pl.spatial(
            adata_vis_0,
            color=[ct],
            spot_size=1,
            cmap="magma",
            size=1.3,
            show=False,
            save=f"{simplified_name}_{ct}_abundance.pdf",
        )

    print(f"Figures and results saved for {simplified_name}")

ERROR! Session/line number was not unique in database. History logging moved to new session 292


/bgfs/alee/LO_LAB/Personal/Brent_Schlegel/envs/CITEgeist/lib/python3.10/site-packages/anndata/_core/anndata.py:617: FutureWarning: You are attempting to set `X` to a matrix on a view which has non-unique indices. The resulting `adata.X` will likely not equal the value to which you set it. To avoid this potential issue, please make a copy of the data first. In the future, this operation will throw an error.
  warnings.warn(msg, FutureWarning, stacklevel=1)


Anndata setup with scvi-tools version 1.0.4.

Setup via `Cell2location.setup_anndata` with arguments:

{
│   'layer': None,
│   'batch_key': None,
│   'labels_key': None,
│   'categorical_covariate_keys': None,
│   'continuous_covariate_keys': None
}

...

               Data Registry                
┏━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Registry Key ┃    scvi-tools Location    ┃
┡━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│      X       │          adata.X          │
│    batch     │ adata.obs['_scvi_batch']  │
│    ind_x     │   adata.obs['_indices']   │
│    labels    │ adata.obs['_scvi_labels'] │
└──────────────┴───────────────────────────┘

                     batch State Registry                      
┏━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┓
┃     Source Location      ┃ Categories ┃ scvi-tools Encoding ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━┩
│ adata.obs['_scvi_batch'] │     0      │          0          │
└──────────────────────────┴────────────┴─────────────────────┘

                     labels State Registry                      
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┓
┃      Source Location      ┃ Categories ┃ scvi-tools Encoding ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━┩
│ adata.obs['_scvi_labels'] │     0      │          0          │
└───────────────────────────┴────────────┴─────────────────────┘

/bgfs/alee/LO_LAB/Personal/Brent_Schlegel/envs/CITEgeist/lib/python3.10/site-packages/scvi/train/_trainrunner.py:76: UserWarning: `use_gpu` is deprecated in v1.0 and will be removed in v1.1. Please use `accelerator` and `devices` instead.
  accelerator, lightning_devices, device = parse_device_args(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/bgfs/alee/LO_LAB/Personal/Brent_Schlegel/envs/CITEgeist/lib/python3.10/site-packages/lightning/pytorch/trainer/configuration_validator.py:69: UserWarning: You passed in a `val_dataloader` but have no `validation_step`. Skipping val loop.
  rank_zero_warn("You passed in a `val_dataloader` but have no `validation_step`. Skipping val loop.")
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
SLURM auto-requeueing enabled. Setting signal handlers.
/bgfs/alee/LO_LAB/Personal/Brent_Schlegel/envs/CITEgeist/lib/python3.10/site-packages/lightning/pytorch/loo

Epoch 1683/30000:   6%|▌         | 1682/30000 [01:40<28:24, 16.62it/s, v_num=1, elbo_train=3.08e+8]